In [3]:
from catinous.dataset import BatchDataset, ContinuousDataset
from catinous.dynamicmemory.DynamicMemoryModel import DynamicMemoryModel 
import catinous.dynamicmemory.DynamicMemoryModel as dmodel
import catinous.utils as cutils
import os
import pandas as pd

In [13]:
hparams = {
    'continuous': False,
    'datasetfile': '/project/catinous/cardiacslicessplit.csv',
    'batch_size': 16,
    'training_batch_size': 16,
    'task': 'cardiac',
    'model': 'fcn',
    'scanner': 'Canon',
     'noncontinuous_steps': 500,
    'noncontinuous_train_splits': ['base', 'train'],
    'use_memory': False
}

#model, logs, df_cache = catsmodel.trained_model(hparams)
expparams = cutils.default_params(DynamicMemoryModel.get_default_hparams(), hparams)
exp_name = cutils.get_expname(expparams)

max_version = max([int(x.split('_')[1]) for x in os.listdir(cutils.LOGGING_FOLDER + exp_name)])
logs = pd.read_csv(cutils.LOGGING_FOLDER + exp_name + '/version_{}/metrics.csv'.format(max_version))

In [14]:
exp_name

'cardiac_batch_cardiacslicessplit_base-train_1_063f1b25d2'

In [15]:
logs

,created_at,epoch,train_loss,val_dice1_Canon,val_dice1_GE,val_dice1_Philips,val_dice1_Siemens,val_dice2_Canon,val_dice2_GE,val_dice2_Philips,val_dice2_Siemens,val_dice3_Canon,val_dice3_GE,val_dice3_Philips,val_dice3_Siemens
0,2021-01-04 07:40:55.700058,0.0,0.141675,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-01-04 08:01:45.444133,0.0,0.083908,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-01-04 08:26:55.966417,NaN,NaN,0.813481,0.618511,0.592816,0.499708,0.699948,0.496320,0.428538,0.376741,0.696748,0.583192,0.417128,0.473839
3,2021-01-04 08:46:06.084374,0.0,0.059129,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-01-04 09:06:51.299302,0.0,0.052579,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2021-01-04 09:32:12.705223,NaN,NaN,0.850412,0.736358,0.689406,0.657063,0.748185,0.600061,0.503194,0.482947,0.768880,0.649380,0.514623,0.564934
6,2021-01-04 09:51:25.828531,0.0,0.039131,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2021-01-04 10:11:56.576528,0.0,0.031734,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
